In [1]:
import os
from PIL import Image
import cv2

import numpy as np
from random import shuffle
import torch
import torchvision
from torchvision import transforms

from tqdm import tqdm

import imgaug as ia
from imgaug import augmenters as iaa

import matplotlib.pyplot as plt
%matplotlib inline

os.environ['CUDA_VISIBLE_DEVICES'] = "0"
DATA_FOLDER="/data/seanyu/cat_dog/dataset/"
EPOCHS = 50

In [2]:
trp = os.path.join(DATA_FOLDER, 'train')
train_path = [os.path.join(trp, i) for i in os.listdir(trp)]

trp = os.path.join(DATA_FOLDER, 'test1')
test_path = [os.path.join(trp, i) for i in os.listdir(trp)]

In [3]:
class DataGenerator():
    """Class for Data Loader"""
    def __init__(self, file_list, transform=None, is_training=True):
        """
        Args:
            - file_path: list of image file path
            - transform: optional transform to be used
        """
        self.file_list = file_list
        self.transform = transform
        self.is_training = is_training
        self.index = 0
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        idx = idx % len(self.file_list)
        img = cv2.cvtColor(cv2.imread(self.file_list[idx]), cv2.COLOR_BGR2RGB)
        #sample = {'image': img}
        sample = img
        
        if self.transform:
            sample = self.transform(sample)
            
        if self.is_training:
            label = ('cat' in os.path.basename(self.file_list[idx])) * 1
        else:
            label = None
            
        return sample, label
    
    def __iter__(self):
        return self

    def __next__(self):
        if self.index == len(self):
            shuffle(self.file_list)
            self.index = 0
        self.index += 1
        return self.__getitem__(self.index-1)

In [4]:
class Imgaug_transform():
    def __init__(self):
        self.aug = iaa.Sequential([iaa.Scale((224,224)),
                                   iaa.Sometimes(0.5, iaa.AddToHueAndSaturation(value=(-15, 15), per_channel=True) ),
                                   iaa.Fliplr(0.5),
                                   iaa.Affine(rotate=(-15, 15), mode='wrap')
                                   ])
    def __call__(self, img):
        img = self.aug.augment_image(img)
        img = img.astype(np.float32) / 255.
        return img
    
imgaug_tra = Imgaug_transform()

py_tra = transforms.Compose([
    Imgaug_transform(),
    transforms.ToTensor()
])

In [5]:
# torch model
from torch import nn, optim
import torch.nn.functional as F

class Create_pytorch_model(nn.Module):
    def __init__(self):
        super(Create_pytorch_model, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1, stride=1)        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1, stride=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1, stride=1)
        self.fc1 = nn.Linear(in_features=64, out_features=2)
        
    def forward(self, x):
        
        x = F.max_pool2d(F.relu(self.conv1(x)), kernel_size=2)
        x = F.max_pool2d(F.relu(self.conv2(x)), kernel_size=2)
        x = F.adaptive_avg_pool2d(F.relu(self.conv3(x)), (1,1))
        
        x = x.view(x.shape[0], -1)
        x = F.log_softmax(self.fc1(x), dim=1)
        return x    

In [6]:
model = Create_pytorch_model()

In [7]:
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
loss_fn = nn.NLLLoss() # cross-entropy, but in single-value encoding (non-onehot)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)

# Check the model is in training mode
model.train()

cuda


Create_pytorch_model(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=64, out_features=2, bias=True)
)

In [9]:
# Pytorch training loop
trainset = DataGenerator(file_list=train_path, transform=py_tra, is_training=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, num_workers=0, shuffle=True)

history_loss = []
history_acc = []
for epoch in range(EPOCHS):
    epoch_loss = []
    running_acc = []
    for image, label in tqdm(trainloader):
        y_pred = model(image.cuda())
        
        loss = loss_fn(y_pred, label.cuda())
        epoch_loss.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        this_acc = sum(y_pred.argmax(dim=1).cpu().numpy() == label.numpy()) / len(y_pred)
        running_acc.append(this_acc) 
    print("EPOCH: %i/%i\tloss: %.4f\taccuracy: %.4f" % (epoch+1, EPOCHS, np.mean(epoch_loss), np.mean(running_acc)))
    history_loss.append(np.mean(epoch_loss))
    history_acc.append(np.mean(running_acc))

  0%|          | 0/391 [00:00<?, ?it/s]

EPOCH: 1	loss: 0.6917	accuracy: 0.5246


  0%|          | 0/391 [00:00<?, ?it/s]

EPOCH: 2	loss: 0.6872	accuracy: 0.5487


  0%|          | 0/391 [00:00<?, ?it/s]

...

EPOCH: 48	loss: 0.5058	accuracy: 0.7527


  0%|          | 0/391 [00:00<?, ?it/s]

EPOCH: 49	loss: 0.5125	accuracy: 0.7532


100%|██████████| 391/391 [02:33<00:00,  2.54it/s]

EPOCH: 50	loss: 0.5109	accuracy: 0.7547
